# Target Percent Pipeline Algorithm

利用pipeline提供的買賣清單與持股權重進行定期再平衡的演算法。

<span id="menu"></span>
本文件包含以下部份：

**基本設定**
1. [Set Environment Variables](#Env)
2. [Investment Universe](#Universe)
3. [Ingest](#Ingest)
4. [Imports](#Imports)
5. [Pipeline](#Pipeline)

**參數說明與範例**

6. [API Reference](#APIReference)：參數說明。
   - [class zipline.algo.pipeline_algo.TargetPercentPipeAlgo](#APIReference)
   - [class zipline.api.date_rules](#date_rules)


7. [Examples](#Examples)：範例。
   - [Case 1－調整start_session與end_session](#Case1)
   - [Case 2－調整max_leverage](#Case2)
   - [Case 3－調整tradeday](#Case3)
   - [Case 4－調整rebalance_date_rule](#Case4)
   - [Case 5－調整slippage_model](#Case5)
   - [Case 6－調整stocklist](#Case6)
   - [Case 7－調整allow_short](#Case7)
   - [Case 8－調整cancel_datedelta](#Case8)   
   - [Case 9－調整limit_buy_multiplier](#Case9)  
   - [Case 10－調整custom_weight、analyze、record_vars、get_record_vars與get_transaction_detail](#Case10)   

<span id="Env"></span>
# 1. Set Environment Variables
[Return to Menu](#menu)

In [1]:
import pandas as pd
import datetime
import tejapi
import os

# set tej_key and base
tej_key = 'YOUR KEY'
api_base = 'https://api.tej.com.tw'

os.environ['TEJAPI_KEY'] = tej_key
os.environ['TEJAPI_BASE']= api_base

tejapi.ApiConfig.api_key = tej_key  
tejapi.ApiConfig.api_base = api_base


# set benchmark
benchmark=['IR0001']

# set calendar
calendar_name='TEJ_XTAI'

# set bundle name
bundle_name = 'tquant'

# set date
start='2023-06-01'
end='2023-10-03'

# 由文字型態轉為Timestamp，供回測使用
tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)

# 設定os.environ['mdate'] = start+' '+end，供ingest bundle使用
os.environ['mdate'] = start+' '+end

<span id="Universe"></span>
# 2. Investment Universe

台灣50指數成分股

[Return to Menu](#menu)

In [2]:
from zipline.sources.TEJ_Api_Data import get_universe

StockList = get_universe(start, end, idx_id='IX0002')

print(len(StockList))

55


In [3]:
os.environ['ticker'] = ' '.join(StockList + benchmark)

In [4]:
os.environ['ticker']

'1101 1216 1301 1303 1326 1402 1590 1605 2002 2207 2301 2303 2308 2317 2327 2330 2345 2357 2379 2382 2395 2408 2412 2454 2603 2609 2615 2801 2880 2881 2882 2883 2884 2885 2886 2887 2890 2891 2892 2912 3008 3034 3037 3045 3231 3711 4904 4938 5871 5876 5880 6415 6505 6669 9910 IR0001'

<span id="Ingest"></span>
# 3. Ingest
[Return to Menu](#menu)

In [5]:
# Ingest pricing bundle
!zipline ingest -b tquant

Merging daily equity files:


[2023-11-09 11:16:26.105989] INFO: zipline.data.bundles.core: Ingesting tquant.


In [6]:
from zipline.data.data_portal import get_bundle_price

df_bundle_price = get_bundle_price(bundle_name=bundle_name,
                                   calendar_name=calendar_name,
                                   start_dt=start_dt,
                                   end_dt=end_dt)

<span id="Imports"></span>
# 4. Imports & Settings
[Return to Menu](#menu)

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
from time import time
import numpy as np
import pandas as pd
import empyrical as ep

from logbook import Logger, StderrHandler, INFO, WARNING
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


from TejToolAPI.TejToolAPI import *

from zipline.api import record

from zipline.utils.calendar_utils import get_calendar
from zipline.utils.run_algo import  (get_transaction_detail,
                                     get_record_vars)

from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.filters import SingleAsset
from zipline.pipeline.factors import RSI
from zipline.pipeline.data import TQDataSet

from zipline.finance import slippage

from zipline.TQresearch.tej_pipeline import run_pipeline

# 設定log顯示方式
log_handler = StderrHandler(format_string='[{record.time:%Y-%m-%d %H:%M:%S.%f}]: ' +
                            '{record.level_name}: {record.func_name}: {record.message}',
                            level=INFO)
log_handler.push_application()
log = Logger('Algorithm')

In [9]:
import warnings
warnings.filterwarnings('ignore')

<span id="Pipeline"></span>
# 5. Pipeline

取得`Market_Cap_Dollars`（市值）資料

[Return to Menu](#menu)

In [10]:
from zipline.pipeline.loaders.fundamentals import TQuantFundamentalsPipelineLoader

In [11]:
col = ['Market_Cap_Dollars']

fields = ''
for i in col:
    fields += i
    fields += ' '

os.environ['fields'] = fields

In [12]:
# Ingest Fundamental Bundle
!zipline ingest -b fundamentals

[2023-11-09 11:16:32.655276] INFO: zipline.data.bundles.core: Ingesting fundamentals.


<span id="APIReference"></span>
# 6. API Reference
[Return to Menu](#menu)

### **class zipline.algo.pipeline_algo.<font color=DeepPink>TargetPercentPipeAlgo</font>** 
*(self, bundle_name='tquant', start_session=None, end_session=None, trading_calendar=get_calendar('TEJ_XTAI'),*

*capital_base=1e7, data_frequency='daily', tradeday=None, rebalance_date_rule=None, stocklist=None, benchmark='IR0001',* 

*zero_treasury_returns=True, slippage_model=slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.1),*

*commission_model=commission.Custom_TW_Commission(min_trade_cost=20, discount = 1.0, tax = 0.003), max_leverage=0.8,* 

*limit_buy_multiplier=None, limit_sell_multiplier=None, allow_short=False, cancel_datedelta=None, custom_weight=False,*

*custom_loader=None, pipeline=None, analyze=None, record_vars=None, get_record_vars=False, get_transaction_detail=False)*

利用pipeline提供的買賣清單與持股權重進行定期再平衡的演算法。必要參數僅有`pipeline`。


#### Parameters:

- **bundle_name** (*str, optional*)－bundle名稱。預設是 **`'tquant'`**。  


- **start_session** (*pd.Timestamp or datetime, optional*)－回測起始日期。預設是**bundle中最早的資料日期**。  


- **end_session** (*pd.Timestamp or datetime, optional*)－回測結束日期。預設是**bundle中最晚的資料日期**。  


- **trading_calendar** (*TradingCalendar, optional*)－
  - 設置交易日曆。預設是 **`get_calendar('TEJ_XTAI')`**。
  - TradingCalendar：`zipline.utils.calendar_utils.TradingCalendar`。  


- **capital_base** (*float, optional*)－初始資金額度。預設是**一千萬**。  


- **data_frequency** (*{'daily', 'minute'}, optional*)－資料頻率，目前僅支援日頻率 **`'daily'`**。  


- **tradeday** (*list[str] or list[pd.Timestamp] or pd.DatetimeIndex, optional*)－
  - 交易日期清單，限制只能在這個清單中的日期進行交易。預設是**None**，代表**每日**都交易。
  - `rebalance_date_rule`與`tradeday`請擇一設定，若兩者皆設定，則會以`rebalance_date_rule`為主。  


- **rebalance_date_rule**(*EventRule, optional*)－
  - 交易頻率，設定固定的交易頻率。預設是**None**，代表**每日**都交易。
  - `rebalance_date_rule`與`tradeday`請擇一設定，若兩者皆設定，則會以`rebalance_date_rule`為主。
  - EventRule：`zipline.utils.events.EventRule`。可使用的`rebalance_date_rule`參考[zipline.api.date_rules](#date_rules)。  


- **stocklist** (*list[str], optional*)－交易清單，限制只能交易這個清單中的股票。預設是**None**，代表使用所有bundle中的股票。  


- **benchmark** (*str, optional*)－指數名稱，用來與投資組合報酬率比較。預設是`'IR0001'`，代表**台灣發行量加權股價報酬指數**。  


- **zero_treasury_returns** (*bool, optional*)－
  - 是否將無風險利率設定為0，預設是**True**，代表設定為0。因此使用`algo.run()`產出的回測報表中，`treasury_return`與`treasury_period_return`皆會是0。
  - 若設定為**False**，則會<font color=DeepPink>耗費額外API流量</font>，並取得第一銀行一年期定存利率作為無風險利率。  


- **slippage_model** (*SlippageModel, optional*)－
  - 設定滑價模型。預設為`slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.1)`。可使用的模型參考：[lecture/Zipline Slippage.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/Zipline%20Slippage.ipynb)
  - SlippageModel：`zipline.finance.slippage.SlippageModel`。


- **commission_model** (*CommissionModel, optional*)－
  - 設定手續費模型。預設為`commission.Custom_TW_Commission(min_trade_cost=20, discount=1.0, tax = 0.003)`。可使用的模型參考：[lecture/Zipline Commission Models.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/Zipline%20Commission%20Models.ipynb)
  - CommissionModel：`zipline.finance.commission.CommissionModel`


- **max_leverage** (*float, optional*)－槓桿限制，預設 = **0.8**。  


- **limit_buy_multiplier** (*float, optional*)－
  - 買進／回補時的limit_price乘數，若有提供則limit_price = 下單時最近一筆收盤價 * `limit_buy_multiplier`。
  - 預設為**None**，代表**不設定**買進／回補時的limit_price。  


- **limit_sell_multiplier** (*float, optional*)－
  - 賣出／放空時的limit_price乘數，若有提供則limit_price = 下單時最近一筆收盤價 * `limit_sell_multiplier`。
  - 預設為**None**，代表**不設定**賣出／放空時的limit_price。  


- **allow_short** (*bool, optional*)－是否允許放空股票，預設為**False**，代表僅能做多。若設定為**True**，則pipeline中需要有`shorts`欄位。  


- **cancel_datedelta** (*int, optional*)－訂單幾天內未完全成交就取消。預設是在**下一次再平衡**時取消。 


- **custom_weight** (*bool, optional*)－
  - 是否要使用自訂的加權權數，預設為**False**，代表不使用（即等權重加權）。
  - 若設定為**True**，則pipeline中需要有`long_weights`（若`allow_short`=True，則也須有`short_weights`）欄位。


- **custom_loader** (*PipelineLoader , optional*)－
  - 用來取得價量以外資料，預設是**None**，代表不使用價量和`TQDataSet`以外資料。
  - TQDataSet：`zipline.pipeline.data.TQDataSet`
  - 目前支援的`PipelineLoader`：
    - DataFrameLoader（`zipline.pipeline.loaders.frame.DataFrameLoader`）。


- **pipeline** (*Pipeline*)－
  - 要用來產出交易清單或權重的pipeline，為<font color=DeepPink>**必要參數**</font>。
  - Pipeline：`zipline.pipeline.Pipeline`


- **analyze** (*callable[(context, pd.DataFrame) -> None], optional*)－
  - 傳入`analyze`函式以用於回測，函式中必須要有`context`與`perf`參數，預設是**None**。
  - 此函式在**回測結束**時被一次性呼叫，並繪製自訂圖表。


- **record_vars** (*callable[(context, BarData) -> None], optional*)－
  - 傳入`record_vars`函式以用於回測，函式中必須要有`context`與`data`參數，預設是**None**。
  - 此函式在**每個交易日結束**時被呼叫，並把指定資料紀錄於回測結果的DataFrame中。


- **get_record_vars** (*bool, optional*)－
  - 是否產出`record_vars`中`record`方法所記錄的變數，預設為**False**，代表不產出。
  - 若設定為**True**，則可用`algo.dict_record_vars`取出。


- **get_transaction_detail** (*bool, optional*)－
  - 是否產出交易結果，預設為**False**，代表不產出。
  - 若設定為**True**，則可用`algo.positions`、`algo.transactions`、`algo.orders`方式取出交易結果。

#### Returns:
    algo

#### Return type:
    zipline.algo.pipeline_algo.TargetPercentPipeAlgo


<br>




<br/>

**<font color=DeepPink>run</font>()**
>
> 執行演算法
> 
> **Returns:**
> - perf（回測報表）
> 
> **Return type:**
> - pd.DataFrame

<span id="date_rules"></span>
### **class zipline.api.<font color=DeepPink>date_rules</font>**
[Return to Menu](#menu)

- 為Factory API，主要用來傳入`zipline.api.schedule_function`的`date_rule`參數中。用來決定要以何種頻率觸發某項規則。
- 使用前請先import：`from zipline.api import date_rules`。
---

*static* **<font color=DeepPink>every_day</font>**()
> 
> 每日觸發某項規則。
> 
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule

<br>

*static* **<font color=DeepPink>month_end</font>**(days_offset=0)
> 
> 每個月底觸發某項規則，並可以選擇性的新增一個偏移量。
> 
> **Parameters:**
> - **days_offset**(*int , optional*)：
>   - 在月底之前的第幾個交易日（由0開始計算）觸發某項規則。預設值是0，即在月底的最後一個交易日觸發。
>   - days_offset只能介於0~22之間。 
> 
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule


<br>

*static* **<font color=DeepPink>month_start</font>**(days_offset=0)
> 
> 每個月初觸發某項規則，並可以選擇性的新增一個偏移量。
> 
> **Parameters:**
> - **days_offset**(*int , optional*)：
>   - 在月初之後的第幾個交易日（由0開始計算）才觸發某項規則。預設值是0，即在月初的第一個交易日觸發。
>   - days_offset只能介於0~22之間。 
> 
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule


<br>

*static* **<font color=DeepPink>week_end</font>**(days_offset=0)
> 
> 在每周最後一個交易日觸發某項規則，並可以選擇性的新增一個偏移量。
> 
> **Parameters:**
> - **days_offset**(*int , optional*)：
>   - 在每周倒數第幾個交易日（由0開始計算）觸發某項規則。預設值是0，即在每周的最後一個交易日觸發。
>   - days_offset只能介於0~4之間。 
>
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule


<br>

*static* **<font color=DeepPink>week_start</font>**(days_offset=0)
> 
> 在每周的第一個交易日觸發某項規則，並可以選擇性的新增一個偏移量。
> 
> **Parameters:**
> - **days_offset**(*int , optional*)：
>   - 在每周的第幾個交易日（由0開始計算）才觸發某項規則。預設值是0，即在每周的第一個交易日觸發。
>   - days_offset只能介於0~4之間。
> 
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule

<span id="Examples"></span>
# 7. Examples
[Return to Menu](#menu)

In [13]:
from zipline.utils.algo_instance import get_algo_instance, set_algo_instance
from zipline.algo.pipeline_algo import TargetPercentPipeAlgo

<span id="Case1"></span>
## Case 1 調整start_session與end_session

[Return to Menu](#menu)

僅調整`start_session`與`end_session`。其餘保持預設值。

以下設定pipeline（`make_pipeline()`），並定義`longs`欄位用來判斷須持有的股票。在`longs`欄位中要持有的股票標記為True，反之標記為False。

In [14]:
from zipline.data import bundles

bundle = bundles.load(bundle_name)

def make_pipeline():
    rsi = RSI()
    longs = rsi.top(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))

    return Pipeline(
        
        columns = {
            "longs" : longs,
        }
    )

In [15]:
algo_start = '2023-09-22'
algo_start_dt = pd.Timestamp(algo_start, tz = tz)

result = run_pipeline(make_pipeline(), algo_start, end)
result.query('longs == True')

longs
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(24 [2603])   True
2023-09-25 00:00:00+00:00 Equity(25 [2609])   True
                          Equity(34 [2886])   True
2023-09-26 00:00:00+00:00 Equity(33 [2885])   True
                          Equity(34 [2886])   True
2023-09-27 00:00:00+00:00 Equity(25 [2609])   True
                          Equity(34 [2886])   True
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(34 [2886])   True
2023-10-02 00:00:00+00:00 Equity(24 [2603])   True
                          Equity(34 [2886])   True
2023-10-03 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(14 [2327])   True

### 執行演算法
1. 實體化`TargetPercentPipeAlgo`並命名為`algo`。
2. 設定演算法：`set_algo_instance(algo)`
3. 執行演算法，並產出回測報表`stats`（*pd.DataFrame*）：`stats = algo.run()`

In [16]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:36.567885]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-11-09 11:16:36.568883]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 982.73, last_sale_price: 981.0
[2023-11-09 11:16:36.568883]: INFO: handle_split: returning cash: 642.32
[2023-11-09 11:16:36.605042]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


查看演算法中的參數設定

In [17]:
algo

TargetPercentPipeAlgo(
    sim_params = 
SimulationParameters(
    start_session=2023-09-22 00:00:00+00:00,
    end_session=2023-10-03 00:00:00+00:00,
    capital_base=10000000.0,
    data_frequency=daily,
    emission_rate=daily,
    first_open=2023-09-22 01:01:00+00:00,
    last_close=2023-10-03 05:30:00+00:00,
    trading_calendar=<exchange_calendars.exchange_calendar_tejxtai.TEJ_XTAIExchangeCalendar object at 0x000001EE4587E460>
),
    benchmark = IR0001,
    zero treasury returns or not（if "True" then treasury returns = 0）= True,
    max_leverage = 0.8,
    slippage model used = VolumeShareSlippage(
    volume_limit=0.025,
    price_impact=0.1),
    commission_model = Custom_TW_Commission(min_trade_cost=20.0),
    adjust amount or not = False,
    limit_buy_multiplier = None,
    limit_sell_multiplier = None,
    allow short or not（if "False" then long only）= False,
    use custom weight or not（if not then "equal weighted"）= False,
    cancel_datedelta（if "None" then cancel open o

In [18]:
stats.T

,2023-09-22 13:30:00+08:00,2023-09-25 13:30:00+08:00,2023-09-26 13:30:00+08:00,2023-09-27 13:30:00+08:00,2023-09-28 13:30:00+08:00,2023-10-02 13:30:00+08:00,2023-10-03 13:30:00+08:00
period_open,2023-09-22 09:01:00+08:00,2023-09-25 09:01:00+08:00,2023-09-26 09:01:00+08:00,2023-09-27 09:01:00+08:00,2023-09-28 09:01:00+08:00,2023-10-02 09:01:00+08:00,2023-10-03 09:01:00+08:00
period_close,2023-09-22 13:30:00+08:00,2023-09-25 13:30:00+08:00,2023-09-26 13:30:00+08:00,2023-09-27 13:30:00+08:00,2023-09-28 13:30:00+08:00,2023-10-02 13:30:00+08:00,2023-10-03 13:30:00+08:00
gross_leverage,0.0,0.80533,0.811325,0.804392,0.799253,0.81946,0.79374
positions,[],"[{'sid': Equity(24 [2603]), 'amount': 34934, '...","[{'sid': Equity(25 [2609]), 'amount': 86950, '...","[{'sid': Equity(34 [2886]), 'amount': 103880, ...","[{'sid': Equity(34 [2886]), 'amount': 103830, ...","[{'sid': Equity(34 [2886]), 'amount': 103418, ...","[{'sid': Equity(34 [2886]), 'amount': 102435, ..."
net_leverage,0.0,0.80533,0.811325,0.804392,0.799253,0.81946,0.79374
starting_exposure,0.0,0.0,8044033.0,7974977.2,7871691.95,7790367.0,7890426.8
ending_exposure,0.0,8044033.0,7974977.2,7871691.95,7790367.0,7890426.8,7606384.75
starting_value,0.0,0.0,8044033.0,7974977.2,7871691.95,7790367.0,7890426.8
ending_value,0.0,8044033.0,7974977.2,7871691.95,7790367.0,7890426.8,7606384.75
orders,"[{'id': '4880ea93c9964812a7f345429f3911cf', 'd...","[{'id': '4880ea93c9964812a7f345429f3911cf', 'd...","[{'id': '2dd7b7a794b6479da46c839d7af4b722', 'd...","[{'id': 'b7952096c8f54cf0a9a20bb99b470a63', 'd...","[{'id': '8f97178e6d8c4483bf3a1f2b86cb77e2', 'd...","[{'id': '889631b485b2497f8b540fc6300af1a5', 'd...","[{'id': '449d7e8398364cd290fd11c365a0e2a8', 'd..."


In [19]:
positions, transactions, orders = get_transaction_detail(stats)

In [20]:
transactions

,sid,symbol,amount,dt,price,order_id,asset,commission
2023-09-25 13:30:00+08:00,24,2603,34934,2023-09-25 13:30:00+08:00,116.000161,4880ea93c9964812a7f345429f3911cf,Equity(24 [2603]),None
2023-09-25 13:30:00+08:00,6,1590,4069,2023-09-25 13:30:00+08:00,981.008992,e887ce5c74e643698b29ed90c0ffdca8,Equity(6 [1590]),None
2023-09-26 13:30:00+08:00,24,2603,-34934,2023-09-26 13:30:00+08:00,115.499883,2dd7b7a794b6479da46c839d7af4b722,Equity(24 [2603]),None
2023-09-26 13:30:00+08:00,6,1590,-4067,2023-09-26 13:30:00+08:00,957.984622,41c85b7972ee4213bb50ce7c2cfb3f2e,Equity(6 [1590]),None
2023-09-26 13:30:00+08:00,25,2609,86950,2023-09-26 13:30:00+08:00,45.950386,c582866e230a499bb971cdf2af37bc7a,Equity(25 [2609]),None
2023-09-26 13:30:00+08:00,34,2886,105142,2023-09-26 13:30:00+08:00,37.850329,60a0a9158dfb4b73aad4ef2e95da1a85,Equity(34 [2886]),None
2023-09-27 13:30:00+08:00,25,2609,-86950,2023-09-27 13:30:00+08:00,45.899835,b7952096c8f54cf0a9a20bb99b470a63,Equity(25 [2609]),None
2023-09-27 13:30:00+08:00,33,2885,157273,2023-09-27 13:30:00+08:00,25.150271,c4d919ae150948068e7ac93e8671b70f,Equity(33 [2885]),None
2023-09-27 13:30:00+08:00,34,2886,-1262,2023-09-27 13:30:00+08:00,37.700000,cca0aa5f907540f88bd5bb431d8a0004,Equity(34 [2886]),None
2023-09-28 13:30:00+08:00,33,2885,-157273,2023-09-28 13:30:00+08:00,25.049463,8f97178e6d8c4483bf3a1f2b86cb77e2,Equity(33 [2885]),None


In [21]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),34934,116.165473,116.00,4052344.00
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),4069,982.407124,981.00,3991689.00
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),86950,46.015872,45.95,3995352.50
2023-09-26 13:30:00+08:00,34,2886,Equity(34 [2886]),105142,37.904275,37.85,3979624.70
2023-09-27 13:30:00+08:00,34,2886,Equity(34 [2886]),103880,37.906306,37.70,3916276.00
2023-09-27 13:30:00+08:00,33,2885,Equity(33 [2885]),157273,25.186113,25.15,3955415.95
2023-09-28 13:30:00+08:00,34,2886,Equity(34 [2886]),103830,37.906557,37.70,3914391.00
2023-09-28 13:30:00+08:00,25,2609,Equity(25 [2609]),85280,45.515097,45.45,3875976.00
2023-10-02 13:30:00+08:00,34,2886,Equity(34 [2886]),103418,37.907234,37.60,3888516.80
2023-10-02 13:30:00+08:00,6,1590,Equity(6 [1590]),3982,1006.440578,1005.00,4001910.00


In [22]:
stats.net_leverage

2023-09-22 13:30:00+08:00    0.000000
2023-09-25 13:30:00+08:00    0.805330
2023-09-26 13:30:00+08:00    0.811325
2023-09-27 13:30:00+08:00    0.804392
2023-09-28 13:30:00+08:00    0.799253
2023-10-02 13:30:00+08:00    0.819460
2023-10-03 13:30:00+08:00    0.793740
Name: net_leverage, dtype: float64

<span id="Case2"></span>
## Case 2 調整max_leverage
[Return to Menu](#menu)

接續**Case 1**，多調整`max_leverage=0.70`，其餘與**Case 1**相同。

In [23]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     max_leverage=0.70,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:36.786696]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 47843.0356576
[2023-11-09 11:16:36.787696]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 3558, cost_basis: 982.73, last_sale_price: 981.0
[2023-11-09 11:16:36.787696]: INFO: handle_split: returning cash: 807.83
[2023-11-09 11:16:36.825616]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [24]:
stats.net_leverage

2023-09-22 13:30:00+08:00    0.000000
2023-09-25 13:30:00+08:00    0.704522
2023-09-26 13:30:00+08:00    0.708474
2023-09-27 13:30:00+08:00    0.703439
2023-09-28 13:30:00+08:00    0.698986
2023-10-02 13:30:00+08:00    0.715863
2023-10-03 13:30:00+08:00    0.694093
Name: net_leverage, dtype: float64

<span id="Case3"></span>
## Case 3 調整tradeday
[Return to Menu](#menu)  

接續**Case 1**，多新增`tradeday`，其餘與**Case 1**相同。

In [25]:
# 設定再平衡日期
freq = 'MS'   # QS-JUL  MS W
_tradeday = list(pd.date_range(start=start_dt, end=end_dt, freq=freq))
tradeday = [get_calendar(calendar_name).next_open(pd.Timestamp(i)).strftime('%Y-%m-%d') if \
           get_calendar(calendar_name).is_session(i)==False else i.strftime('%Y-%m-%d') for i in _tradeday]
tradeday

['2023-06-01', '2023-07-03', '2023-08-01', '2023-09-01', '2023-10-02']

In [26]:
algo_start_dt

Timestamp('2023-09-22 00:00:00+0000', tz='UTC')

In [27]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,           
                     tradeday=tradeday,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:36.942625]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [28]:
positions, transactions, orders = get_transaction_detail(stats)

In [29]:
transactions

,sid,symbol,amount,dt,price,order_id,asset,commission
2023-10-03 13:30:00+08:00,24,2603,35242,2023-10-03 13:30:00+08:00,110.500051,358932c7b3344e42a2ee6a4416b71237,Equity(24 [2603]),None
2023-10-03 13:30:00+08:00,34,2886,106382,2023-10-03 13:30:00+08:00,37.650179,a5c402492b524ef9baabec4883df855d,Equity(34 [2886]),None


<span id="Case4"></span>
## Case 4 調整rebalance_date_rule
[Return to Menu](#menu)  

接續**Case 1**，多新增`rebalance_date_rule`，並修改`start_session`為2023-09-01，其餘與**Case 1**相同。

In [30]:
from zipline.api import date_rules

algo = TargetPercentPipeAlgo(
                     start_session=pd.Timestamp('2023-09-01', tz='UTC'),
                     end_session=end_dt,
                     pipeline=make_pipeline,
                     # 每月的第四個交易日下單
                     rebalance_date_rule=date_rules.month_start(days_offset=3) 
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:37.090660]: INFO: handle_simulation_end: Simulated 22 trading days
first open: 2023-09-01 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [31]:
positions, transactions, orders = get_transaction_detail(stats)

In [32]:
orders

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-06 13:30:00+08:00,41,3034,6adca04061db48f5874b8ad5ddea345e,2023-09-06 13:30:00+08:00,None,2023-09-06 13:30:00+08:00,9411,0,0,None,None,False,False,Equity(41 [3034]),0
2023-09-06 13:30:00+08:00,14,2327,d14b52eece68431e9c3f5f3e508ef8f4,2023-09-06 13:30:00+08:00,None,2023-09-06 13:30:00+08:00,7920,0,0,None,None,False,False,Equity(14 [2327]),0
2023-09-07 13:30:00+08:00,41,3034,6adca04061db48f5874b8ad5ddea345e,2023-09-07 13:30:00+08:00,None,2023-09-06 13:30:00+08:00,9411,9411,5727,None,None,False,False,Equity(41 [3034]),1
2023-09-07 13:30:00+08:00,14,2327,d14b52eece68431e9c3f5f3e508ef8f4,2023-09-07 13:30:00+08:00,None,2023-09-06 13:30:00+08:00,7920,7920,5666,None,None,False,False,Equity(14 [2327]),1


<span id="Case5"></span>
## Case 5 調整slippage_model
[Return to Menu](#menu)  

接續**Case 1**，多新增`slippage_model`，將`volume_limit`由**0.025**調整為**0.01**，其餘與**Case 1**相同。

In [33]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     pipeline=make_pipeline,
                     slippage_model=slippage.VolumeShareSlippage(volume_limit=0.01, price_impact=0.1)
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:37.197623]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-11-09 11:16:37.198623]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 982.73, last_sale_price: 981.0
[2023-11-09 11:16:37.199147]: INFO: handle_split: returning cash: 642.32
[2023-11-09 11:16:37.203327]: INFO: rebalance: Cancel_order: current time: 2023-09-26 , created: 2023-09-26 , asset: Equity(6 [1590]), amount: -857 , filled: 0
[2023-11-09 11:16:37.210684]: INFO: rebalance: Cancel_order: current time: 2023-09-27 , created: 2023-09-26 , asset: Equity(33 [2885]), amount: 157331 , filled: 151500
[2023-11-09 11:16:37.213857]: INFO: rebalance: Cancel_order: current time: 2023-09-28 , created: 2023-09-28 , asset: Equity(33 [2885]), amount: -44120 , filled: 0
[2023-11-09 11:16:37.229268]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-

In [34]:
positions, transactions, orders = get_transaction_detail(stats)

In [35]:
orders.query('(symbol == "1590") & (created.dt.strftime("%Y-%m-%d") == "2023-09-25")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-25 13:30:00+08:00,6,1590,7df20632bacd45f0b9b1cf3f2f627ab8,2023-09-25 13:30:00+08:00,None,2023-09-25 13:30:00+08:00,-4069,0,0.0,None,None,False,False,Equity(6 [1590]),0
2023-09-26 13:30:00+08:00,6,1590,7df20632bacd45f0b9b1cf3f2f627ab8,2023-09-26 13:30:00+08:00,None,2023-09-25 13:30:00+08:00,-4067,-3210,13609.0,None,None,False,False,Equity(6 [1590]),2


In [36]:
# 321000 * 1% = 3210(股) 

df_bundle_price.query('(symbol == "1590") & (date.dt.strftime("%Y-%m-%d") == "2023-09-26")')[['symbol','date','volume']]

,symbol,date,volume
4486,1590,2023-09-26 00:00:00+00:00,321000.0


In [37]:
orders.query('(symbol == "2885") & (created.dt.strftime("%Y-%m-%d") == "2023-09-27")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-27 13:30:00+08:00,33,2885,abab2db1c29e4a1b8efeac42ecad9c4a,2023-09-27 13:30:00+08:00,None,2023-09-27 13:30:00+08:00,-151500,0,0.0,None,None,False,False,Equity(33 [2885]),0
2023-09-28 13:30:00+08:00,33,2885,abab2db1c29e4a1b8efeac42ecad9c4a,2023-09-28 13:30:00+08:00,None,2023-09-27 13:30:00+08:00,-151500,-107380,11904.0,None,None,False,False,Equity(33 [2885]),2


In [38]:
# 10738000 * 1% = 107380(股) 

df_bundle_price.query('(symbol == "2885") & (date.dt.strftime("%Y-%m-%d") == "2023-09-28")')[['symbol','date','volume']]

,symbol,date,volume
4625,2885,2023-09-28 00:00:00+00:00,10738000.0


<span id="Case6"></span>
## Case 6 調整stocklist
[Return to Menu](#menu)  


接續**Case 1**，多新增`stocklist`，其餘與**Case 1**相同。  



註1：`stocklist`限制是在pipeline執行完後。  

註2：也可以使用pipeline直接限制股票池。

In [39]:
len(StockList)

55

In [40]:
_StockList = [i for i in StockList if i!='2886']
len(_StockList)

54

In [41]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,            
                     stocklist=_StockList,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:37.482574]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-11-09 11:16:37.483574]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 982.73, last_sale_price: 981.0
[2023-11-09 11:16:37.483574]: INFO: handle_split: returning cash: 642.32
[2023-11-09 11:16:37.514729]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [42]:
positions, transactions, orders = get_transaction_detail(stats)

In [43]:
positions

,sid,symbol,asset,amount,cost_basis,last_sale_price
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),34934,116.165473,116.00
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),4069,982.407124,981.00
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),86950,46.015872,45.95
2023-09-27 13:30:00+08:00,33,2885,Equity(33 [2885]),157364,25.186112,25.15
2023-09-28 13:30:00+08:00,25,2609,Equity(25 [2609]),85469,45.515096,45.45
2023-10-02 13:30:00+08:00,6,1590,Equity(6 [1590]),3991,1006.440643,1005.00
2023-10-03 13:30:00+08:00,24,2603,Equity(24 [2603]),34046,110.657511,110.50


<span id="Case7"></span>
## Case 7 調整allow_short
[Return to Menu](#menu) 

接續**Case 1**，多新增`allow_short=True`，其餘與**Case 1**相同。  


以下設定pipeline（`make_pipeline()`），並定義`shorts`欄位用來判斷須放空的股票。在`shorts`欄位中要放空的股票標記為True，反之標記為False。

In [44]:
def make_pipeline():
    rsi = RSI()
    longs = rsi.top(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))
    shorts = rsi.bottom(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))

    return Pipeline(
        columns = {
            "longs" : longs,
            "shorts" : shorts
        }
    )

In [45]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [46]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     allow_short=True,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:37.685701]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-11-09 11:16:37.686875]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 982.73, last_sale_price: 981.0
[2023-11-09 11:16:37.686875]: INFO: handle_split: returning cash: 642.32
[2023-11-09 11:16:37.718687]: INFO: rebalance: Cancel_order: current time: 2023-10-02 , created: 2023-09-28 , asset: Equity(39 [2912]), amount: -14713 , filled: -8575
[2023-11-09 11:16:37.730813]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [47]:
positions, transactions, orders = get_transaction_detail(stats)

In [48]:
# 當天取消
orders.query('(symbol == "2912") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,39,2912,ba92e168398146db9396be4d63842879,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,0,0.0,None,None,False,False,Equity(39 [2912]),0
2023-10-02 13:30:00+08:00,39,2912,ba92e168398146db9396be4d63842879,2023-10-02 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,-8575,9980.0,None,None,False,False,Equity(39 [2912]),2


In [49]:
# 343000 * 2.5% = 8575(股) 

df_bundle_price.query('(symbol == "2912") & (date.dt.strftime("%Y-%m-%d") == "2023-10-02")')[['symbol','date','volume']]

,symbol,date,volume
4687,2912,2023-10-02 00:00:00+00:00,343000.0


In [50]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions.query('(symbol == "2912")')

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-25 13:30:00+08:00,39,2912,Equity(39 [2912]),-15151,263.315516,264.5,-4007439.5
2023-10-02 13:30:00+08:00,39,2912,Equity(39 [2912]),-8575,261.819714,263.0,-2255225.0
2023-10-03 13:30:00+08:00,39,2912,Equity(39 [2912]),-14195,261.431485,262.0,-3719090.0


<span id="Case8"></span>
## Case 8 調整cancel_datedelta
[Return to Menu](#menu) 

接續**Case 7**，多新增`cancel_datedelta=2`，其餘與**Case 7**相同。  

In [51]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     allow_short=True,
                     cancel_datedelta=2,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:37.935559]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-11-09 11:16:37.936559]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 982.73, last_sale_price: 981.0
[2023-11-09 11:16:37.936559]: INFO: handle_split: returning cash: 642.32
[2023-11-09 11:16:37.988552]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [52]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [53]:
positions, transactions, orders = get_transaction_detail(stats)

In [54]:
orders.query('(symbol == "2912") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,39,2912,bc7e5ee1313a45d1955c9b536787701c,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,0,0.0,None,None,False,False,Equity(39 [2912]),0
2023-10-02 13:30:00+08:00,39,2912,bc7e5ee1313a45d1955c9b536787701c,2023-10-02 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,-8575,9980.0,None,None,False,False,Equity(39 [2912]),0
2023-10-03 13:30:00+08:00,39,2912,bc7e5ee1313a45d1955c9b536787701c,2023-10-03 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,-14713,17097.0,None,None,False,False,Equity(39 [2912]),1


In [55]:
# 10/02：343000 * 2.5% = 8575(股) 
# 10/03：808000 * 2.5% = 20200(股) 

df_bundle_price.query('(symbol == "2912") & (date.dt.strftime("%Y-%m-%d")>="2023-10-02")')\
                 [['symbol','date','volume']]

,symbol,date,volume
4687,2912,2023-10-02 00:00:00+00:00,343000.0
4743,2912,2023-10-03 00:00:00+00:00,808000.0


In [56]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions.query('(symbol == "2912")')

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-25 13:30:00+08:00,39,2912,Equity(39 [2912]),-15151,263.315516,264.5,-4007439.5
2023-10-02 13:30:00+08:00,39,2912,Equity(39 [2912]),-8575,261.819714,263.0,-2255225.0
2023-10-03 13:30:00+08:00,39,2912,Equity(39 [2912]),-14713,261.410574,262.0,-3854806.0


<span id="Case9"></span>
## Case 9 調整limit_buy_multiplier
[Return to Menu](#menu) 

接續**Case 8**，多設定`limit_buy_multiplier=1.015`，其餘與**Case 8**相同。

In [57]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [58]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     limit_buy_multiplier=1.015,
                     allow_short=True,
                     cancel_datedelta=2,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:38.206957]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-11-09 11:16:38.207430]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 982.73, last_sale_price: 981.0
[2023-11-09 11:16:38.208671]: INFO: handle_split: returning cash: 642.32
[2023-11-09 11:16:38.257992]: INFO: exec_cancel: Cancel_order: current time: 2023-10-03,
                              due to : created>=current time + cancel_datedelta(2 days),
                              created: 2023-09-28, asset: Equity(6 [1590]), amount: 3945, filled: 0'
[2023-11-09 11:16:38.263235]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [59]:
positions, transactions, orders = get_transaction_detail(stats)

In [60]:
orders

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-22 13:30:00+08:00,24,2603,dbddae4914874ce48c785c07c701cd5f,2023-09-22 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,34934,0,0.0,None,116.21,False,False,Equity(24 [2603]),0
2023-09-22 13:30:00+08:00,6,1590,3e52fe36e6f7473182e1b75af08bbcf3,2023-09-22 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,4069,0,0.0,None,997.74,False,False,Equity(6 [1590]),0
2023-09-22 13:30:00+08:00,10,2301,bd6cddb1e8024b05ad420dbceab62db1,2023-09-22 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,-33333,0,0.0,None,NaN,False,False,Equity(10 [2301]),0
2023-09-22 13:30:00+08:00,39,2912,815b22d181734fc2a05a9621f7791c68,2023-09-22 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,-15151,0,0.0,None,NaN,False,False,Equity(39 [2912]),0
2023-09-25 13:30:00+08:00,24,2603,dbddae4914874ce48c785c07c701cd5f,2023-09-25 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,34934,34934,5775.0,None,116.21,False,True,Equity(24 [2603]),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-03 13:30:00+08:00,39,2912,6a719572c9094197ae4323508ebd6fce,2023-10-03 13:30:00+08:00,None,2023-10-03 13:30:00+08:00,14713,0,0.0,None,NaN,False,False,Equity(39 [2912]),0
2023-10-03 13:30:00+08:00,6,1590,bec5658638ea43a19e083c830b48653b,2023-10-03 13:30:00+08:00,None,2023-10-03 13:30:00+08:00,3776,0,0.0,None,1012.97,False,False,Equity(6 [1590]),0
2023-10-03 13:30:00+08:00,14,2327,6d1eaa13074241aea687c84f12f811a2,2023-10-03 13:30:00+08:00,None,2023-10-03 13:30:00+08:00,7247,0,0.0,None,527.80,False,False,Equity(14 [2327]),0
2023-10-03 13:30:00+08:00,2,1301,98b48fc6654847d88975edab876c7abe,2023-10-03 13:30:00+08:00,None,2023-10-03 13:30:00+08:00,-47642,0,0.0,None,NaN,False,False,Equity(2 [1301]),0


In [61]:
orders.query('(symbol == "1590") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,6,1590,aca40f1cfa54489faf88f73ebdfa219c,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,3945,0,0.0,None,993.68,False,False,Equity(6 [1590]),0
2023-10-03 13:30:00+08:00,6,1590,aca40f1cfa54489faf88f73ebdfa219c,2023-10-03 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,3945,0,0.0,None,993.68,False,False,Equity(6 [1590]),2


In [62]:
# 9/28 979 * 1.015 = 993.685
df_bundle_price.query('(symbol == "1590") & (date.dt.strftime("%Y-%m-%d")>="2023-09-28")')\
                 [['symbol','date','close']]

,symbol,date,close
4598,1590,2023-09-28 00:00:00+00:00,979.0
4654,1590,2023-10-02 00:00:00+00:00,1005.0
4710,1590,2023-10-03 00:00:00+00:00,998.0


<span id="Case10"></span>
## Case 10 調整custom_weight、analyze、record_vars、get_record_vars與get_transaction_detail
[Return to Menu](#menu) 

接續**Case 9**，多設定`custom_weight`=True、`analyze`、`record_vars`、`get_record_vars`=True與`get_transaction_detail`=True，其餘與**Case 9**相同。

In [63]:
class Weight(CustomFactor):
    
    inputs =  [TQDataSet.Market_Cap_Dollars] 
    outputs = ["Market_Cap_Dollars","Sum_Market_Cap_Dollars","Weight"]   
    window_length = 1

    def compute(self, today, assets, out, Market_Cap_Dollars):
        
        out.Market_Cap_Dollars[:] = Market_Cap_Dollars
        out.Sum_Market_Cap_Dollars[:] = np.nansum(Market_Cap_Dollars, axis=1)
        out.Weight[:] = Market_Cap_Dollars / np.sum(Market_Cap_Dollars, axis=1)

In [64]:
def make_pipeline():
    
    rsi = RSI()
    longs = rsi.top(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))
    shorts = rsi.bottom(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))

    return Pipeline(
        
        columns = {
            "Market_Cap_Dollars":Weight().Market_Cap_Dollars,        
            "longs" : longs,
            "shorts" : shorts,
            "long_weights" : Weight(mask=longs).Weight,
            "short_weights" : Weight(mask=shorts).Weight
        }
    )

In [65]:
def analyze(context, perf):
    
    fig = plt.figure(figsize=(16, 24), dpi=400)
    
    # First chart(累積報酬)
    ax = fig.add_subplot(611) 
    ax.set_title('Strategy Results') 
    ax.plot(perf['algorithm_period_return'],
            linestyle='-', 
            label='algorithm period return',
            linewidth=3.0)
    ax.plot(perf['benchmark_period_return'],
            linestyle='-', 
            label='benchmark period return',
            linewidth=3.0)
    ax.legend()
    ax.grid(False)
    
    # Second chart(returns)
    ax = fig.add_subplot(612, sharex=ax)       
    ax.plot(perf['returns'],
            linestyle='-', 
            label='returns',
            linewidth=3.0)
    ax.legend()
    ax.grid(False)

    # Third chart(ending_cash) -> 觀察是否超買
    ax = fig.add_subplot(613, sharex=ax)
    ax.plot(perf['ending_cash'], 
            label='ending_cash',
            linestyle='-',
            linewidth=3.0)
    ax.axhline(y=1, c='r', linewidth=1)
    ax.legend()
    ax.grid(False)

    # Forth chart(shorts_count) -> 觀察是否放空
    ax = fig.add_subplot(614, sharex=ax)
    ax.plot(perf['shorts_count'], 
            label='shorts_count',
            linestyle='-',
            linewidth=3.0)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.axhline(y=0, c='r', linewidth=1)
    ax.legend()
    ax.grid(False)
    
    # Fifth chart(longs_count)
    ax = fig.add_subplot(615, sharex=ax)
    ax.plot(perf['longs_count'], 
            label='longs_count',
            linestyle='-',
            linewidth=3.0)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.axhline(y=0, c='r', linewidth=1)
    ax.legend()
    ax.grid(False) 
    
    # Sixth chart(weights) -> 觀察每日持股權重
    ax = fig.add_subplot(616, sharex=ax)        
    weights = pd.concat([df.to_frame(d) for d, df in perf['daily_weights'].dropna().items()],axis=1).T
    
    for i in weights.columns:
        df = weights.loc[:,i]
        ax.scatter(df.index,df.values,marker='.', s=5, c='grey', label='daily_weights')
    ax.axhline(y=0, c='r', linewidth=1)
    ax.legend(['daily_weights'])
    ax.grid(False)

    fig.tight_layout()

def record_vars(context, data):
    """
    Plot variables at the end of each day.
    """
            
    record(daily_weights=context.daily_weights,
           Market_Cap_Dollars=context.output.Market_Cap_Dollars
          )

In [66]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     limit_buy_multiplier=1.015,
                     allow_short=True,
                     custom_weight=True,
                     cancel_datedelta=2,
                     pipeline=make_pipeline,
                     analyze=analyze,
                     record_vars=record_vars,
                     get_record_vars=True,
                     get_transaction_detail=True
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2023-11-09 11:16:38.603136]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 48945.03816432
[2023-11-09 11:16:38.603136]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 3640, cost_basis: 982.73, last_sale_price: 981.0
[2023-11-09 11:16:38.603136]: INFO: handle_split: returning cash: 781.17
[2023-11-09 11:16:38.692575]: INFO: exec_cancel: Cancel_order: current time: 2023-10-03,
                              due to : created>=current time + cancel_datedelta(2 days),
                              created: 2023-09-28, asset: Equity(6 [1590]), amount: 2153, filled: 0'
[2023-11-09 11:16:38.708754]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [67]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

Market_Cap_Dollars  longs  \
2023-09-22 00:00:00+00:00 Equity(6 [1590])         1.980000e+11   True   
                          Equity(10 [2301])        2.818339e+11  False   
                          Equity(24 [2603])        2.444465e+11   True   
                          Equity(39 [2912])        2.739405e+11  False   
2023-09-25 00:00:00+00:00 Equity(10 [2301])        2.842023e+11  False   
                          Equity(25 [2609])        1.599384e+11   True   
                          Equity(31 [2883])        1.970197e+11  False   
                          Equity(34 [2886])        5.311406e+11   True   
2023-09-26 00:00:00+00:00 Equity(8 [2002])         4.122534e+11  False   
                          Equity(10 [2301])        2.842023e+11  False   
                          Equity(33 [2885])        3.197649e+11   True   
                          Equity(34 [2886])        5.339508e+11   True   
2023-09-27 00:00:00+00:00 Equity(10 [2301])        2.842023e+11  False   
                          Equity(17 [2357])        2.659082e+11  False   
                          Equity(25 [2609])        1.604622e+11   True   
                          Equity(34 [2886])        5.318431e+11   True   
2023-09-28 00:00:00+00:00 Equity(6 [1590])         1.982000e+11   True   
                          Equity(10 [2301])        2.842023e+11  False   
                          Equity(34 [2886])        5.297354e+11   True   
                          Equity(39 [2912])        2.734207e+11  False   
2023-10-02 00:00:00+00:00 Equity(10 [2301])        2.877548e+11  False   
                          Equity(24 [2603])        2.455047e+11   True   
                          Equity(34 [2886])        5.297354e+11   True   
                          Equity(39 [2912])        2.729008e+11  False   
2023-10-03 00:00:00+00:00 Equity(2 [1301])         5.067130e+11  False   
                          Equity(6 [1590])         2.010000e+11   True   
                          Equity(14 [2327])        2.255366e+11   True   
                          Equity(30 [2882])        6.527799e+11  False   

                                             shorts  long_weights  \
2023-09-22 00:00:00+00:00 Equity(6 [1590])    False      0.447512   
                          Equity(10 [2301])    True           NaN   
                          Equity(24 [2603])   False      0.552488   
                          Equity(39 [2912])    True           NaN   
2023-09-25 00:00:00+00:00 Equity(10 [2301])    True           NaN   
                          Equity(25 [2609])   False      0.231433   
                          Equity(31 [2883])    True           NaN   
                          Equity(34 [2886])   False      0.768567   
2023-09-26 00:00:00+00:00 Equity(8 [2002])     True           NaN   
                          Equity(10 [2301])    True           NaN   
                          Equity(33 [2885])   False      0.374557   
                          Equity(34 [2886])   False      0.625443   
2023-09-27 00:00:00+00:00 Equity(10 [2301])    True           NaN   
                          Equity(17 [2357])    True           NaN   
                          Equity(25 [2609])   False      0.231780   
                          Equity(34 [2886])   False      0.768220   
2023-09-28 00:00:00+00:00 Equity(6 [1590])    False      0.272277   
                          Equity(10 [2301])    True           NaN   
                          Equity(34 [2886])   False      0.727723   
                          Equity(39 [2912])    True           NaN   
2023-10-02 00:00:00+00:00 Equity(10 [2301])    True           NaN   
                          Equity(24 [2603])   False      0.316682   
                          Equity(34 [2886])   False      0.683318   
                          Equity(39 [2912])    True           NaN   
2023-10-03 00:00:00+00:00 Equity(2 [1301])     True           NaN   
                          Equity(6 [1590])    False      0.471237   
                          Equity(14

#### algo.positions

In [68]:
# 計算實際股票部位的weight = 個股持股市值／所有股票部位的持股市值加總
pos = algo.positions

pos['mv'] = pos['amount'] * pos['last_sale_price']

positive_sum = pos[pos['mv'] > 0].groupby(level=0)['mv'].sum()
negative_sum = - pos[pos['mv'] < 0].groupby(level=0)['mv'].sum()
pos['sum'] = np.where(pos['mv'] > 0,
                            pos.index.map(positive_sum),
                            pos.index.map(negative_sum))

pos['weight'] = pos['mv'] / pos['sum']
pos

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv,sum,weight
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),38601,116.165504,116.00,4477716.00,8050518.00,0.556202
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),3642,982.405337,981.00,3572802.00,8050518.00,0.443798
2023-09-25 13:30:00+08:00,10,2301,Equity(10 [2301]),-33806,119.468850,120.00,-4056720.00,8007292.00,-0.506628
2023-09-25 13:30:00+08:00,39,2912,Equity(39 [2912]),-14936,263.315870,264.50,-3950572.00,8007292.00,-0.493372
2023-09-26 13:30:00+08:00,10,2301,Equity(10 [2301]),-39186,119.468865,120.00,-4702320.00,7948441.00,-0.591603
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),40103,46.015564,45.95,1842732.85,7938096.85,0.232138
2023-09-26 13:30:00+08:00,34,2886,Equity(34 [2886]),161040,37.904715,37.85,6095364.00,7938096.85,0.767862
2023-09-26 13:30:00+08:00,31,2883,Equity(31 [2883]),-275095,11.747584,11.80,-3246121.00,7948441.00,-0.408397
2023-09-27 13:30:00+08:00,10,2301,Equity(10 [2301]),-26645,119.388362,120.00,-3197400.00,7728416.20,-0.413720
2023-09-27 13:30:00+08:00,34,2886,Equity(34 [2886]),129471,37.945396,37.70,4881056.70,7833364.90,0.623111


#### algo.dict_record_vars

In [69]:
record_vars = algo.dict_record_vars

In [70]:
# 實際持股市值 = 個股持股市值／所有股票部位的持股市值加總 * max_leverage
record_vars['daily_weights']

,date,symbol,daily_weights
0,2023-09-25 00:00:00+08:00,2603,0.449893
1,2023-09-25 00:00:00+08:00,1590,0.358973
2,2023-09-25 00:00:00+08:00,2301,-0.407594
3,2023-09-25 00:00:00+08:00,2912,-0.396929
4,2023-09-26 00:00:00+08:00,2301,-0.480126
5,2023-09-26 00:00:00+08:00,2609,0.188150
6,2023-09-26 00:00:00+08:00,2886,0.622361
7,2023-09-26 00:00:00+08:00,2883,-0.331442
8,2023-09-27 00:00:00+08:00,2301,-0.328375
9,2023-09-27 00:00:00+08:00,2886,0.501288


In [71]:
# 個股總市值
record_vars['Market_Cap_Dollars']

,date,symbol,Market_Cap_Dollars
0,2023-09-22 00:00:00+08:00,1101,2.418789e+11
1,2023-09-22 00:00:00+08:00,1216,3.966047e+11
2,2023-09-22 00:00:00+08:00,1301,5.168982e+11
3,2023-09-22 00:00:00+08:00,1303,5.313650e+11
4,2023-09-22 00:00:00+08:00,1326,3.745298e+11
...,...,...,...
387,2023-10-03 00:00:00+08:00,6415,1.188855e+11
388,2023-10-03 00:00:00+08:00,6505,7.649346e+11
389,2023-10-03 00:00:00+08:00,6669,2.867389e+11
390,2023-10-03 00:00:00+08:00,9910,1.920654e+11


[Return to Menu](#menu) 